In [ ]:
import pandas as pd
import pytesseract
from PIL import Image
import requests
from io import BytesIO
from tqdm import tqdm  # Import tqdm for progress bar

# Tesseract Path Setup (Windows only, uncomment below line if needed)
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Function to download image and extract text
def extract_text_from_image(image_url):
    try:
        # Download image from the URL
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content))

        # Extract text from image
        extracted_text = pytesseract.image_to_string(img)
        return extracted_text
    except Exception as e:
        return str(e)  # Return the error message if something goes wrong

# Path to the dataset CSV file
input_csv_path = 'dataset/train.csv'

# Read the dataset CSV file
df = pd.read_csv(input_csv_path)

# Prompt the user for the number of entries to process
user_input = input("Enter the number of elements to process (e.g., 50, 100) or type 'all' to process all entries: ")

# Determine the number of entries to process
if user_input.lower() == 'all':
    num_entries = df.shape[0]
else:
    try:
        num_entries = int(user_input)
        if num_entries > df.shape[0]:
            num_entries = df.shape[0]  # Limit to the number of available rows
    except ValueError:
        print("Invalid input. Processing the first 10 entries by default.")
        num_entries = 10

# Create a list to hold results
results_list = []

# Process the specified number of entries with a progress bar
for index, row in tqdm(df.head(num_entries).iterrows(), total=num_entries, desc='Processing Images'):
    image_url = row['image_link']
    extracted_text = extract_text_from_image(image_url)
    
    # Append the result to the list
    results_list.append({
        'group_id': row['group_id'],
        'image_link': image_url,
        'text_ext': extracted_text,
        'entity_name': row['entity_name'],
        'entity_value': row['entity_value']
    })

# Convert the list to a DataFrame
results_df = pd.DataFrame(results_list)

# Path to the output CSV file
output_csv_path = 'tesseract_ocr.csv'

# Save the results to a new CSV file
results_df.to_csv(output_csv_path, index=False)

print(f"OCR text extraction completed and saved to {output_csv_path}")
